In [1]:
!pip install beautifulsoup4

In [49]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pickle

In [3]:
url="https://www.macrotrends.net/stocks/charts/BBY/best-buy/pe-ratio"
    
# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

In [4]:
# Parse the html content
soup = BeautifulSoup(html_content, "html")
print(soup.prettify()) # print the parsed data of html

<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <link href="https://www.macrotrends.net/stocks/charts/BBY/best-buy/pe-ratio" rel="canonical"/>
  <title>
   Best Buy PE Ratio 2006-2020 | BBY | MacroTrends
  </title>
  <meta content="&lt;p&gt;Current and historical p/e ratio for Best Buy (BBY) from 2006 to 2020.  The price to earnings ratio is calculated by taking the latest closing price and dividing it by the most recent earnings per share (EPS) number.  The PE ratio is a simple way to assess whether a stock is over or under valued and is the most widely used valuation measure.  Best Buy PE ratio as of January 13, 2021 is &lt;strong&gt;15.63&lt;/s

In [5]:
print(soup.title)

<title>Best Buy PE Ratio 2006-2020 | BBY | MacroTrends</title>


In [6]:
print(soup.title.text)

Best Buy PE Ratio 2006-2020 | BBY | MacroTrends


In [7]:
ratios_table = soup.find("table", attrs={"class": "table"})

In [8]:
ratios_table_data = ratios_table.find_all("tr")  # contains 2 rows


In [9]:
ratios_table_data

[<tr>
 <th colspan="4" style="text-align:center;">Best Buy PE Ratio Historical Data</th>
 </tr>,
 <tr>
 <th style="text-align:center;">Date</th>
 <th style="text-align:center;">Stock Price</th>
 <th style="text-align:center;">TTM Net EPS</th>
 <th style="text-align:center;">PE Ratio</th>
 </tr>,
 <tr>
 <td style="text-align:center;">2021-01-13</td>
 <td style="text-align:center;">113.22</td>
 <td style="text-align:center;"></td>
 <td style="text-align:center;">17.21</td>
 </tr>,
 <tr>
 <td style="text-align:center;">2020-10-31</td>
 <td style="text-align:center;">110.95</td>
 <td style="text-align:center;">$6.58</td>
 <td style="text-align:center;">16.86</td>
 </tr>,
 <tr>
 <td style="text-align:center;">2020-07-31</td>
 <td style="text-align:center;">98.55</td>
 <td style="text-align:center;">$6.20</td>
 <td style="text-align:center;">15.89</td>
 </tr>,
 <tr>
 <td style="text-align:center;">2020-04-30</td>
 <td style="text-align:center;">75.42</td>
 <td style="text-align:center;">$5.4

In [10]:
len(ratios_table_data)

64

In [11]:
ratios_table_data[4]

<tr>
<td style="text-align:center;">2020-07-31</td>
<td style="text-align:center;">98.55</td>
<td style="text-align:center;">$6.20</td>
<td style="text-align:center;">15.89</td>
</tr>

In [12]:
# Get all the headings of Lists
headings = []
for th in ratios_table_data[1].find_all("th"):
    # remove any newlines and extra spaces from left and right
    headings.append(th.text.replace('\n', ' ').strip())

print(headings)

['Date', 'Stock Price', 'TTM Net EPS', 'PE Ratio']


In [13]:
l_elems=list()
df_ratios=pd.DataFrame()

for row in ratios_table_data:
    l_elems=list()
    for td in row.find_all("td"):
        elem=td.text
        l_elems.append(elem)
    df_ratios = df_ratios.append(pd.Series(l_elems), ignore_index=True)   
    
df_ratios.columns=headings

<ipython-input-13-2e4dc774087c>:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_ratios = df_ratios.append(pd.Series(l_elems), ignore_index=True)


In [14]:
df_ratios

,Date,Stock Price,TTM Net EPS,PE Ratio
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,2021-01-13,113.22,,17.21
3,2020-10-31,110.95,$6.58,16.86
4,2020-07-31,98.55,$6.20,15.89
...,...,...,...,...
59,2007-11-30,36.09,0,0.00
60,2007-08-31,30.98,0,0.00
61,2007-05-31,33.97,0,0.00
62,2007-02-28,32.63,0,0.00


In [15]:
df2=df_ratios.copy()

In [16]:
df_ratios

,Date,Stock Price,TTM Net EPS,PE Ratio
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,2021-01-13,113.22,,17.21
3,2020-10-31,110.95,$6.58,16.86
4,2020-07-31,98.55,$6.20,15.89
...,...,...,...,...
59,2007-11-30,36.09,0,0.00
60,2007-08-31,30.98,0,0.00
61,2007-05-31,33.97,0,0.00
62,2007-02-28,32.63,0,0.00


In [17]:
df_PE_ratios=df_ratios[['Date', 'PE Ratio']]
df_PE_ratios.set_index('Date')

df_PE_ratios= df_PE_ratios.rename(columns={'PE Ratio': 'AAPL'})

In [18]:
df_PE_ratios

,Date,AAPL
0,NaN,NaN
1,NaN,NaN
2,2021-01-13,17.21
3,2020-10-31,16.86
4,2020-07-31,15.89
...,...,...
59,2007-11-30,0.00
60,2007-08-31,0.00
61,2007-05-31,0.00
62,2007-02-28,0.00


# Automatizado

In [19]:
companies_tickers = ['AAPL',
            'AMZN',
            'MSFT',
            'GOOG',
            'FB',
            'NVDA',
             #automobiles
            'F',
            'GM',
            'HMC',
            'TSLA',
            'TYO',
             'HOG',
             #retail
             'WMT',
             'M',
             'TGT',
             'KSS',
             'COST',
             'BBY',
             #FastMovingConsumerGoods(FMCG)
             'NESN',
             'KO',
             'PEP',
             'ULVR',
             'PG',
             'OR',
            #bank
            'AXP',
            'VISA',
            'SC',
            'DB',
            'UBS',
            'BAC',
            'JPM']

In [20]:
companies_names=[
#tech
'apple',
'amazon',
'microsoft',
'google',
'facebook',
'nvidia',
#automobiles
'ford',
'general-motors',
'honda',
'tesla',
'toyota',
'harley-davidson',
#retail
'walmart',
'macys',
'target',
'kohl',
'costco',
'best-buy',
#FastMovingConsumerGoods(FMCG)
'nestle',
'cocacola',
'pepsi',
'unilever',
'procter-gamble',
'loreal',
 #bank
'american-express',
'visa',
'santander-consumer-usa-holdings',
'deutsche-bank-aktiengesellschaft',
'ubs-group-ag',
'bank-of-america',
'jpmorgan-chase']

In [21]:
# Get all the headings of Lists
headings = []
for th in ratios_table_data[1].find_all("th"):
    # remove any newlines and extra spaces from left and right
    headings.append(th.text.replace('\n', ' ').strip())

print(headings)

['Date', 'Stock Price', 'TTM Net EPS', 'PE Ratio']


In [55]:
ratios_collection = {} 

for company_ticker, company_name in zip(companies_tickers, companies_names):
    url="https://www.macrotrends.net/stocks/charts/" + company_ticker  +"/"+ company_name + "/pe-ratio"
    print(url)
    # Make a GET request to fetch the raw HTML content
    html_content = requests.get(url).text

    # Parse the html content
    soup = BeautifulSoup(html_content, "html")

    try:
        #Search for table
        ratios_table = soup.find("table", attrs={"class": "table"})
        #Get rows
        ratios_table_data = ratios_table.find_all("tr")

        # Get all the headings of Lists
        headings = []
        for th in ratios_table_data[1].find_all("th"):
            # remove any newlines and extra spaces from left and right
            h=th.text.replace('\n', ' ').strip()
            #replace spaces with underscores
            h=h.replace(" ", "_")
            headings.append(h)


        #Extract and collect elements from table.
        l_elems=list()
        df_ratios=pd.DataFrame()

        for row in ratios_table_data:
            l_elems=list()
            cont=0
            for td in row.find_all("td"):
                cont+=1

                if cont==1:
                    elem=td.text[:7]
#                     if elem[5:] in ['01','02','03','04']:
#                         quarter='Q1'
#                         elem=elem[:4]+quarter
#                     elif elem[5:] in ['05','06','07','08']:
#                         quarter='Q2'
#                         elem=elem[:4]+quarter
#                     elif elem[5:] in ['09','10','11','12']:
#                         quarter='Q3'
#                         elem=elem[:4]+quarter
                else:
                    elem=td.text

                l_elems.append(elem)
            df_ratios = df_ratios.append(pd.Series(l_elems), ignore_index=True)   
        cont=0

        df_ratios.columns=headings
        df_ratios['Date'] = pd.to_datetime(df_ratios['Date'])
        ratios_collection[company_ticker] = df_ratios

    except:
        print("Couldnt retrieve info from   " + company_ticker + "..." + company_name)


https://www.macrotrends.net/stocks/charts/AAPL/apple/pe-ratio


<ipython-input-55-23d340e1b0d5>:53: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_ratios = df_ratios.append(pd.Series(l_elems), ignore_index=True)


https://www.macrotrends.net/stocks/charts/AMZN/amazon/pe-ratio
https://www.macrotrends.net/stocks/charts/MSFT/microsoft/pe-ratio
https://www.macrotrends.net/stocks/charts/GOOG/google/pe-ratio
https://www.macrotrends.net/stocks/charts/FB/facebook/pe-ratio
https://www.macrotrends.net/stocks/charts/NVDA/nvidia/pe-ratio
https://www.macrotrends.net/stocks/charts/F/ford/pe-ratio
https://www.macrotrends.net/stocks/charts/GM/general-motors/pe-ratio
https://www.macrotrends.net/stocks/charts/HMC/honda/pe-ratio
https://www.macrotrends.net/stocks/charts/TSLA/tesla/pe-ratio
https://www.macrotrends.net/stocks/charts/TYO/toyota/pe-ratio
Couldnt retrieve info from   TYO...toyota
https://www.macrotrends.net/stocks/charts/HOG/harley-davidson/pe-ratio
https://www.macrotrends.net/stocks/charts/WMT/walmart/pe-ratio
https://www.macrotrends.net/stocks/charts/M/macys/pe-ratio
https://www.macrotrends.net/stocks/charts/TGT/target/pe-ratio
https://www.macrotrends.net/stocks/charts/KSS/kohl/pe-ratio
https://www.m

In [52]:
#         df_PE_ratios=df_ratios[['Date', 'PE Ratio']]
#         #si dejo el Date como index, tendré un monton de rows porque no todas las fechas son iguales
# #         df_PE_ratios=df_PE_ratios.set_index('Date')
#         df_PE_ratios= df_PE_ratios.rename(columns={'PE Ratio': company_name})

#         if companies_tickers.index(company_ticker)==0:
#             df_PE_ratios_final=df_PE_ratios.copy()
#         else:
#             df_PE_ratios_final=pd.merge(df_PE_ratios_final, df_PE_ratios, on ='Date' )
# #             df_PE_ratios_final=df_PE_ratios_final.join(df_PE_ratios)


In [57]:
ratios_collection['DB'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         54 non-null     datetime64[ns]
 1   Stock_Price  54 non-null     object        
 2   TTM_Net_EPS  54 non-null     object        
 3   PE_Ratio     54 non-null     object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.9+ KB


In [58]:
pickle.dump(ratios_collection, open('ratios_collection.p', "wb" ))


In [ ]:

try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen

import json

def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url)
    data = response.read().decode("utf-8")
    print(data)
    return json.loads(data)

url = ("https://financialmodelingprep.com/api/v3/key-metrics/AAPL?apikey=d5864a708b9191f7f178fc759a4d6f4b")
print(url)
print(get_jsonparsed_data(url))

In [ ]:
PE_ratios=pd.DataFrame()


In [ ]:
#tengo 250 peticiones al dia
for comp in companies:
    print(comp)
    url = ("https://financialmodelingprep.com/api/v3/key-metrics/" + comp + "?apikey=d5864a708b9191f7f178fc759a4d6f4b")
    print(url)
    prueba=get_jsonparsed_data(url)
    
    if len(prueba)>0:
        if prueba[0]['symbol']==comp:
            for i in range(0,len(prueba)):
                PE_ratios.loc[prueba[i]['date'][:4], comp]=prueba[i]['peRatio']